In [0]:
# ============================================================
# 1. Imports
# ============================================================

import unicodedata
import re
from functools import reduce

from pyspark.sql import DataFrame
from pyspark.sql.functions import (
    col, regexp_replace, when, trim,
    min, max, count, lit
)


# ============================================================
# 2. Função: Normalizar nomes das colunas
#    - Remove acentos
#    - Converte para lowercase
#    - Substitui caracteres especiais por "_"
# ============================================================

def normalizar_colunas(df: DataFrame) -> DataFrame:
    novas_colunas = []

    for c in df.columns:
        c_norm = (
            unicodedata.normalize("NFKD", c)
            .encode("ASCII", "ignore")
            .decode("utf-8")
        )
        c_norm = c_norm.lower()
        c_norm = re.sub(r"[^a-z0-9]+", "_", c_norm)
        c_norm = c_norm.strip("_")

        novas_colunas.append(c_norm)

    return df.toDF(*novas_colunas)


# ============================================================
# 3. Leitura das tabelas INMET (2021–2025)
# ============================================================

tabelas = [
    "workspace.default.inmet_ne_se_a_409_aracaju_01_01_2021_a_31_12_2021",
    "workspace.default.inmet_ne_se_a_409_aracaju_01_01_2022_a_31_12_2022",
    "workspace.default.inmet_ne_se_a_409_aracaju_01_01_2023_a_31_12_2023",
    "workspace.default.inmet_ne_se_a_409_aracaju_01_01_2024_a_31_12_2024",
    "workspace.default.inmet_ne_se_a_409_aracaju_01_01_2025_a_31_08_2025",
]

dfs_normalizados = [
    normalizar_colunas(spark.table(tabela))
    for tabela in tabelas
]


# ============================================================
# 4. Consolidação dos DataFrames
#    - Union by name
#    - Permite colunas ausentes
# ============================================================

df_consolidado = reduce(
    lambda a, b: a.unionByName(b, allowMissingColumns=True),
    dfs_normalizados
)


# ============================================================
# 5. Conversão de colunas numéricas
#    - Trata valores inválidos
#    - Converte vírgula para ponto
#    - Cast para double
# ============================================================

colunas_nao_numericas = {"data", "hora_utc"}

def converter_para_double(df: DataFrame) -> DataFrame:
    df_out = df

    for c in df.columns:
        if c not in colunas_nao_numericas:
            df_out = df_out.withColumn(
                c,
                when(
                    col(c).isNull()
                    | (trim(col(c).cast("string")) == "")
                    | (col(c).cast("string").isin("-9999", "////", "NaN")),
                    None
                ).otherwise(
                    regexp_replace(
                        col(c).cast("string"), ",", "."
                    ).cast("double")
                )
            )

    return df_out


df_numerico = converter_para_double(df_consolidado)

# Verificação do schema final
df_numerico.printSchema()


# ============================================================
# 6. Identificação das colunas numéricas
# ============================================================

colunas_numericas = [
    c for c, t in df_numerico.dtypes
    if t not in ("string", "date")
]


# ============================================================
# 7. Análise de qualidade dos dados
#    - Mínimo
#    - Máximo
#    - Dados faltantes
# ============================================================

total_registros = df_numerico.count()

dfs_stats = []

for c in colunas_numericas:
    df_stats = df_numerico.select(
        lit(c).alias("coluna"),
        min(c).alias("minimo"),
        max(c).alias("maximo"),
        (lit(total_registros) - count(c)).alias("dados_faltantes"),
        lit(total_registros).alias("total_registros")
    )
    dfs_stats.append(df_stats)


# ============================================================
# 8. Consolidação da tabela de qualidade
# ============================================================

df_qualidade = reduce(
    lambda a, b: a.unionByName(b),
    dfs_stats
)

display(df_qualidade)
